# Depthwise-seperable 2D Convolution on GPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

# %mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-MaximClouser' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-MaximClouser
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  5-conv2d_dw_gpu.ipynb	README.md  tests
2-conv1d_gpu.ipynb  4-gemm_gpu.ipynb	 leaderboard_id.txt	src


## 2 Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.6/428.6 MB 2.6 MB/s eta 0:00:00


## 3. Implement `make_dwsp_conv2d_gpu_scheduler` function in `src.ops`

In that function, you are required to implemented 2D convolution and use TVM to optimize it.
Please use zero padding and unit stride.
You can assume kernel size to be an odd number.
The padding will equals to kernel size minus ones.
In this case, the output image will preserve the input image dimension.

The `make_dwsp_conv2d_gpu_scheduler` takes following arguments:
1. Batch size $B$;
2. Input channel size $C$;
3. Input image height $H$;
4. Input image width $W$;
5. Output number of channels $O$;
6. Kernel size $K$

You should return both the TVM scheduler and the TVM opterator for
1. Input tensor $x$ with size (B, C, H, W)
2. Input kernel weight $y$ with size (O, 1, K, K)
3. Output $out$ with size (B, O, H, W)

The scheduler should be able to used to build a function with signature $func(x, y, out)$.
Please see the following cells the usage.

In [17]:
!git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 4), reused 7 (delta 3), pack-reused 0
Unpacking objects: 100% (8/8), 722 bytes | 7.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-MaximClouser
   672ff00..38f3e48  main       -> origin/main
Updating 672ff00..38f3e48
Fast-forward
 src/ops.py | 90 +---------------------------------------------------------------------------------
 1 file changed, 1 insertion(+), 89 deletions(-)


In [18]:
import tvm
import torch.nn.functional as F
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_dwsp_conv2d_gpu_scheduler

B = 3
C = 4
H = 16
W = 32
K = 7
dtype = 'float32'
a_np = np.random.rand(B, C, H, W).astype(dtype)
w_np = np.random.rand(C, 1, K, K).astype(dtype)

s, inp, ker, out = make_dwsp_conv2d_gpu_scheduler(B, C, H, W, K)
func = tvm.build(s, [inp, ker, out], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((B, C, H, W), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=50, repeat=5)

print("Output:", b)
print(f"2DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Output: [[[[ 3.8304532  4.803108   5.7486305 ...  6.1292677  3.710828
     3.5066247]
   [ 4.6619697  5.83127    6.6002007 ...  6.093545   5.42578
     4.1304884]
   [ 6.296014   6.157938   8.153242  ...  8.800191   7.2219076
     4.992079 ]
   ...
   [ 5.926825   7.8861012  7.028704  ...  7.7985525  7.3650327
     5.4941573]
   [ 5.1476736  5.831187   6.083035  ...  6.239583   5.5225263
     3.9527605]
   [ 4.923433   4.79796    5.329775  ...  4.5535903  3.5310402
     3.4677918]]

  [[ 3.0828679  3.2280967  3.8403378 ...  4.672493   3.5870113
     3.22329  ]
   [ 3.8048701  4.465809   6.1279955 ...  5.784019   4.0521126
     3.8974159]
   [ 4.42979    6.372895   7.408566  ...  6.534562   6.6234784
     4.1978116]
   ...
   [ 3.869799   6.0715795  7.2491627 ...  7.7713795  7.169382
     5.668649 ]
   [ 3.1004577  4.42315    5.231193  ...  7.220471   6.308419
     4.7295566]
   [ 2.5184689  4.458597   4.8530564 ...  5.9990783  4.204236
     3.8433888]]

  [[ 5.3611274  6.0140233  8.743

In [19]:
print(tvm.lower(s, [inp, ker, out], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(input: T.Buffer((3, 4, 16, 32), "float32"), kernel: T.Buffer((4, 1, 7, 7), "float32"), output: T.Buffer((3, 4, 16, 32), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_y = T.launch_thread("blockIdx.y", 12)
        blockIdx_x = T.launch_thread("blockIdx.x", 1)
        threadIdx_y = T.env_thread("threadIdx.y")
        output_1 = T.Buffer((6144,), data=output.data)
        with T.launch_thread(threadIdx_y, 32):
            for w_inner_outer_init in range(8):
                if T.likely(threadIdx_y < 16):
                    output_1[blockIdx_y * 512 + threadIdx_y * 32 + w_inner_outer_init * 4:blockIdx_y * 512 + threadIdx_y * 32 + w_inner_outer_init * 4 + 4] = T.Broadcast(T.float32(0), 4)
        input_1 = T.Buffer((6144,), data=input.data)
        kernel_1 = T.Buffer((196,), data=kernel.data)


In [20]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_dwsp_2dconv_gpu.py

/content/gdrive/MyDrive/ece5545/a3-MaximClouser
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-MaximClouser
plugins: anyio-3.7.1
collected 1357 items                                                                               

tests/test_dwsp_2dconv_gpu.py .............................................................. [  4%]
............................................................................................ [ 11%]
............................................................................................ [ 18%]
............................................................................................ [ 24%]
............................................................................................ [ 31%]
............................................................................................ [ 38%]
...........